In [8]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [9]:
class net(torch.nn.Module):
    def __init__(self, base_model, base_out_features, num_classes):
        super(net,self).__init__()
        self.base_model=base_model
        self.linear1 = torch.nn.Linear(base_out_features, 512)
        self.output = torch.nn.Linear(512,num_classes)
    def forward(self,x):
        x = F.relu(self.base_model(x))
        x = F.relu(self.linear1(x))
        x = self.output(x)
        return x

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

In [11]:
res = torchvision.models.resnet50(pretrained=True)
for param in res.parameters():
    param.requires_grad=False

model_dog = net(base_model=res, base_out_features=res.fc.out_features, num_classes=80)
model_dog = model_dog.to(device)

model_cat = net(base_model=res, base_out_features=res.fc.out_features, num_classes=80)
model_cat = model_cat.to(device)

In [ ]:
model_dog.load_state_dict(torch.load('/content/drive/MyDrive/졸업논문/best_model.pth', map_location=device))
model_dog.eval()

model_cat.load_state_dict(torch.load('/content/drive/MyDrive/졸업논문/copy_model_수정.pth', map_location=device))
model_cat.eval()

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!mkdir 'test_img'

In [ ]:
!unzip /content/drive/MyDrive/졸업논문/data/test_case/dogs_test.zip -d /content/test_img/

In [46]:
test_list = os.listdir('./test_img')
cat_output=[]
dog_output=[]

In [16]:
dir = './test_img'
test_img = {'img':test_list}
test_img = pd.DataFrame(test_img)
test_img.img = test_img.img.apply(lambda x:dir+'/'+x)

In [21]:
dog_df = pd.read_csv('./drive/MyDrive/졸업논문/data/labels_수정.csv')
print('Training set: {}'.format(dog_df.shape[0]))
cat_df = pd.read_csv('./drive/MyDrive/졸업논문/data/고양이/cat_label.csv')
print('Training set: {}'.format(cat_df.shape[0]))

Training set: 6898
Training set: 90079


In [26]:
dog_df['label'] = LabelEncoder().fit_transform(dog_df.breed)
cat_df['label'] = LabelEncoder().fit_transform(cat_df.breed)
dict_dog_df = dog_df[['label','breed']].copy()
dict_cat_df = cat_df[['label','breed']].copy()
dict_dog_df.drop_duplicates(inplace=True)
dict_dog_df.set_index('label',drop=True,inplace=True)
dog_index_to_breed = dict_dog_df.to_dict()['breed']
dict_cat_df.drop_duplicates(inplace=True)
dict_cat_df.set_index('label',drop=True,inplace=True)
cat_index_to_breed = dict_cat_df.to_dict()['breed']

In [32]:
dog_index_to_breed[0]

'affenpinscher'

In [22]:
class img_dataset(Dataset):
    def __init__(self,dataframe,transform=None,test=False):
        self.dataframe = dataframe
        self.transform = transform
        self.test = test
        
    def __getitem__(self,index):
        x = Image.open(self.dataframe.iloc[index,0]).convert('RGB')
        #x = Image.open(self.dataframe.iloc[index,0])
        
        if self.transform:
            x = self.transform(x)
        if self.test:
            return x
        else:
            y = self.dataframe.iloc[index,1]
            return x,y
        
    def __len__(self):
        return self.dataframe.shape[0]

In [23]:
train_transformer = transforms.Compose([transforms.RandomResizedCrop(224),
                                        transforms.RandomRotation(15),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

val_transformer = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [44]:
test_dataset = img_dataset(test_img,transform=val_transformer,test=True)

In [47]:
test_loader = DataLoader(test_dataset)
for x in test_loader:
  x = x.to(device)
  y = model_dog(x)
  yhat = model_cat(x)
  _,predicted_dog = torch.max(y.data, 1)
  _,predicted_cat = torch.max(yhat.data,1)
  predicted_dog = predicted_dog.cpu()
  predicted_cat = predicted_cat.cpu()
  index_dog = predicted_dog.numpy()[0]
  index_cat = predicted_cat.numpy()[0]
  cat_output.append(index_cat)
  dog_output.append(index_dog)

print(len(cat_output))
print(len(dog_output))

4005
4005


In [48]:
output_cat=[]
output_dog=[]
for i in cat_output:
  output_cat.append(cat_index_to_breed[i])
for j in dog_output:
  output_dog.append(dog_index_to_breed[j])

In [49]:
print(output_cat)

['chihuahua', 'schipperke', 'chihuahua', 'pekinese', 'chihuahua', 'norwegian_elkhound', 'labrador_retriever', 'great_pyrenees', 'dandie_dinmont', 'cairn', 'schipperke', 'schipperke', 'chihuahua', 'chihuahua', 'schipperke', 'shetland_sheepdog', 'dingo', 'basset', 'schipperke', 'norwich_terrier', 'keeshond', 'chihuahua', 'chihuahua', 'schipperke', 'shetland_sheepdog', 'french_bulldog', 'schipperke', 'pekinese', 'chihuahua', 'staffordshire_bullterrier', 'german_short-haired_pointer', 'affenpinscher', 'weimaraner', 'samoyed', 'basset', 'german_short-haired_pointer', 'staffordshire_bullterrier', 'staffordshire_bullterrier', 'schipperke', 'shetland_sheepdog', 'chihuahua', 'chihuahua', 'schipperke', 'affenpinscher', 'chihuahua', 'bouvier_des_flandres', 'schipperke', 'chihuahua', 'miniature_schnauzer', 'rhodesian_ridgeback', 'chihuahua', 'toy_terrier', 'schipperke', 'schipperke', 'eskimo_dog', 'greater_swiss_mountain_dog', 'norwegian_elkhound', 'basset', 'chihuahua', 'chihuahua', 'schipperke',

In [50]:
print(output_dog)

['doberman', 'doberman', 'whippet', 'great_pyrenees', 'miniature_pinscher', 'norwegian_elkhound', 'labrador_retriever', 'great_pyrenees', 'old_english_sheepdog', 'airedale', 'labrador_retriever', 'labrador_retriever', 'german_shepherd', 'doberman', 'labrador_retriever', 'bluetick', 'ibizan_hound', 'bluetick', 'american_staffordshire_terrier', 'irish_terrier', 'airedale', 'toy_terrier', 'rottweiler', 'doberman', 'old_english_sheepdog', 'american_staffordshire_terrier', 'doberman', 'english_setter', 'lakeland_terrier', 'german_shepherd', 'bluetick', 'tibetan_terrier', 'weimaraner', 'samoyed', 'bluetick', 'german_short-haired_pointer', 'staffordshire_bullterrier', 'staffordshire_bullterrier', 'staffordshire_bullterrier', 'greater_swiss_mountain_dog', 'english_setter', 'rhodesian_ridgeback', 'rottweiler', 'gordon_setter', 'american_staffordshire_terrier', 'giant_schnauzer', 'labrador_retriever', 'german_short-haired_pointer', 'english_setter', 'rhodesian_ridgeback', 'bluetick', 'bluetick',

In [51]:
correct = 0
for i,j in enumerate(output_cat):
  if j == output_dog[i]:
    correct +=1
print(correct)

701


In [52]:
print(correct/len(output_cat))

0.17503121098626717
